In [1]:
import torch
from transformers import BertTokenizer

from data.dataset import CustomDataset
from torch.utils.data import DataLoader
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

data_loader=DataLoader(CustomDataset(tokenize=True), batch_size=32, shuffle=True)


In [10]:
# number of examples 
print(f"number of examples are: {len(data_loader.dataset)}")

number of examples are: 4012


In [11]:
# sample question-answer examples
import random

Q_A_data=CustomDataset().dataset
random_idx=random.randint(0,len(Q_A_data)-1)
print(f"Question\n {Q_A_data[random_idx]['question']}")
print(f"Answer\n {Q_A_data[random_idx]['answer']}")

Question
 Is SATB1 necessary for T-cell maturation?
Answer
 Special AT-rich sequence binding protein 1 (SATB1) regulates gene expression essential in immune T-cell maturation and switching of fetal globin species, by binding to matrix attachment regions (MARs) of DNA and inducing a local chromatin remodeling.


## Training

In [13]:
from model.model import DualEncoder
embed_dim=512
output_dim=128
epochs=10
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model = DualEncoder(tokenizer,embed_dim,output_dim).to(device)

In [ ]:
model.train(epochs,data_loader=data_loader)

Epoch 1/10


  1%|          | 1/126 [00:12<25:35, 12.29s/it]

Epoch 0, loss = 4.334316730499268


  9%|▊         | 11/126 [04:45<1:07:32, 35.24s/it]

Epoch 0, loss = 4.0125790726054795


...

 40%|████      | 51/126 [32:02<54:02, 43.24s/it]

Epoch 0, loss = 3.7330664653404084


 48%|████▊     | 61/126 [40:05<49:33, 45.75s/it]  

Epoch 0, loss = 3.7023555880687278


 54%|█████▍    | 68/126 [44:25<27:05, 28.02s/it]

In [ ]:
torch.save(model.state_dict(),'model.pt')

In [ ]:
from loss.loss import CrossEntropyLoss
optim=torch.optim.Adam(model.parameters(),lr=1e-5)
criterion=CrossEntropyLoss()
running_loss=[]
for batch,(q,a) in enumerate(data_loader):
    
    
    q = {k: v.to(device) for k, v in q.items()}
    a = {k: v.to(device) for k, v in a.items()}
    _,_,sin_score=model(q,a)
    loss=criterion(sin_score)
    #loss.backward()
    #optim.step()
    running_loss.append(loss.item())
    print(loss.item())
    
    
    print(sin_score)